[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Metaliguns77/Geopypi/blob/main/docs/examples/codliver.ipynb)

In [16]:
import geopypi.common as geopypi
urls = ["https://prd-tnm.s3.amazonaws.com/StagedProducts/Hydrography/NHD/State/GDB/NHD_H_District_of_Columbia_State_GDB.zip"]
geopypi.download_files(urls, out_dir='.', unzip=True)

Downloading...
From: https://prd-tnm.s3.amazonaws.com/StagedProducts/Hydrography/NHD/State/GDB/NHD_H_District_of_Columbia_State_GDB.zip
To: d:\spring 24\software design\Geopypi\docs\examples\NHD_H_District_of_Columbia_State_GDB.zip
100%|██████████| 73.5M/73.5M [00:15<00:00, 4.63MB/s]


Extracting files...


In [1]:
pip install lonboard

Note: you may need to restart the kernel to use updated packages.


In [1]:
import geopypi.common as neo
import geopypi
import lonboard

state = "DC"   # Change to the US State of your choice
url = f"https://data.source.coop/giswqs/nwi/wetlands/{state}_Wetlands.parquet"
gdf = neo.read_parquet(url, return_type='gdf', src_crs='EPSG:5070', dst_crs='EPSG:4326')
geopypi.view_vector(gdf, get_fill_color=[0, 0, 255, 128])

name 'convert_coordinates' is not defined


Map(layers=[SolidPolygonLayer(get_fill_color=[0, 0, 255, 128], table=pyarrow.Table
ATTRIBUTE: string
WETLAND_T…

In [1]:
import duckdb

con = duckdb.connect()
con.install_extension("spatial")
con.load_extension("spatial")

In [2]:
count_df = con.sql(
    f"""
SELECT SUBSTRING(filename, LENGTH(filename) - 18, 2) AS State, COUNT(*) AS Count
FROM read_parquet('s3://us-west-2.opendata.source.coop/giswqs/nwi/wetlands/*.parquet', filename=true)
GROUP BY State
ORDER BY COUNT(*) DESC;
"""
).df()
count_df.head(10)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,State,Count
0,MN,2674597
1,TX,2263259
2,ND,2077099
3,MT,1857721
4,WI,1718865
5,SD,1664305
6,AK,1649192
7,NE,1516060
8,MO,1262141
9,KS,1063234


In [3]:
con.sql("CREATE OR REPLACE TABLE wetlands AS FROM count_df")
con.sql("FROM wetlands")

┌─────────┬─────────┐
│  State  │  Count  │
│ varchar │  int64  │
├─────────┼─────────┤
│ MN      │ 2674597 │
│ TX      │ 2263259 │
│ ND      │ 2077099 │
│ MT      │ 1857721 │
│ WI      │ 1718865 │
│ SD      │ 1664305 │
│ AK      │ 1649192 │
│ NE      │ 1516060 │
│ MO      │ 1262141 │
│ KS      │ 1063234 │
│ ·       │     ·   │
│ ·       │     ·   │
│ ·       │     ·   │
│ NJ      │  248702 │
│ MD      │  240320 │
│ AZ      │  189606 │
│ CT      │  170964 │
│ VT      │  164400 │
│ WV      │  154348 │
│ DE      │   78597 │
│ RI      │   55229 │
│ HI      │   13566 │
│ DC      │    1555 │
├─────────┴─────────┤
│      51 rows      │
│    (20 shown)     │
└───────────────────┘

In [4]:
url = "https://open.gishub.org/data/us/us_states.parquet"
con.sql(
    f"""
CREATE OR REPLACE TABLE states AS
SELECT * EXCLUDE geometry, ST_GeomFromWKB(geometry)
AS geometry FROM '{url}'
"""
)
con.sql("FROM states")

┌─────────┬───────────────┬────────────────────────────────────────────────────────────────────────────────────────────┐
│   id    │     name      │                                          geometry                                          │
│ varchar │    varchar    │                                          geometry                                          │
├─────────┼───────────────┼────────────────────────────────────────────────────────────────────────────────────────────┤
│ AL      │ Alabama       │ POLYGON ((-87.359296 35.00118, -85.606675 34.984749, -85.431413 34.124869, -85.184951 32…  │
│ AK      │ Alaska        │ MULTIPOLYGON (((-131.602021 55.117982, -131.569159 55.28229, -131.355558 55.183705, -131…  │
│ AZ      │ Arizona       │ POLYGON ((-109.042503 37.000263, -109.04798 31.331629, -111.074448 31.331629, -112.24651…  │
│ AR      │ Arkansas      │ POLYGON ((-94.473842 36.501861, -90.152536 36.496384, -90.064905 36.304691, -90.218259 3…  │
│ CA      │ California    │ POLY

In [5]:
con.sql(
    """
SELECT * FROM states INNER JOIN wetlands ON states.id = wetlands.State
"""
)

┌─────────┬───────────────┬────────────────────────────────────────────────────────────────────────┬─────────┬─────────┐
│   id    │     name      │                                geometry                                │  State  │  Count  │
│ varchar │    varchar    │                                geometry                                │ varchar │  int64  │
├─────────┼───────────────┼────────────────────────────────────────────────────────────────────────┼─────────┼─────────┤
│ MN      │ Minnesota     │ POLYGON ((-92.014696 46.705401, -92.091373 46.749217, -92.29402 46.6…  │ MN      │ 2674597 │
│ TX      │ Texas         │ POLYGON ((-101.812942 36.501861, -100.000075 36.501861, -100.000075 …  │ TX      │ 2263259 │
│ ND      │ North Dakota  │ POLYGON ((-97.228743 49.000239, -97.097296 48.682577, -97.16302 48.5…  │ ND      │ 2077099 │
│ MT      │ Montana       │ POLYGON ((-104.047534 49.000239, -104.042057 47.861036, -104.047534 …  │ MT      │ 1857721 │
│ WI      │ Wisconsin     │ POLY

In [6]:
df = con.sql(
    """
SELECT name, State, Count, ST_AsText(geometry) as geometry
FROM states INNER JOIN wetlands ON states.id = wetlands.State
"""
).df()
df.head()

,name,State,Count,geometry
0,Minnesota,MN,2674597,"POLYGON ((-92.014696 46.705401, -92.091373 46...."
1,Texas,TX,2263259,"POLYGON ((-101.812942 36.501861, -100.000075 3..."
2,North Dakota,ND,2077099,"POLYGON ((-97.228743 49.000239, -97.097296 48...."
3,Montana,MT,1857721,"POLYGON ((-104.047534 49.000239, -104.042057 4..."
4,Wisconsin,WI,1718865,"POLYGON ((-90.415429 46.568478, -90.229213 46...."


In [7]:
import geopypi.common as geopypi
gdf = geopypi.df_to_gdf(df, src_crs="EPSG:4326")

In [9]:
import geopypi
m = geopypi.Map()
m.add_data(
    gdf, column='Count', scheme='Quantiles', cmap='Greens', legend_title='Wetland Count'
)
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [11]:
import geopypi.plot as geopypi

geopypi.pie_chart(count_df, 'State', 'Count', height=800, title='Number of Wetlands by State')

In [12]:
geopypi.bar_chart(count_df, 'State', 'Count', title='Number of Wetlands by State')

In [13]:
con.sql(f"""
SELECT SUM(Shape_Area) /  1000000 AS Area_SqKm
FROM 's3://us-west-2.opendata.source.coop/giswqs/nwi/wetlands/*.parquet'
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌────────────────────┐
│     Area_SqKm      │
│       double       │
├────────────────────┤
│ 1442876.9096808708 │
└────────────────────┘

In [14]:
area_df = con.sql(f"""
SELECT SUBSTRING(filename, LENGTH(filename) - 18, 2) AS State, SUM(Shape_Area) /  1000000 AS Area_SqKm
FROM read_parquet('s3://us-west-2.opendata.source.coop/giswqs/nwi/wetlands/*.parquet', filename=true)
GROUP BY State
ORDER BY COUNT(*) DESC;
""").df()
area_df.head(10)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,State,Area_SqKm
0,MN,66376.803485
1,TX,43200.379897
2,ND,17586.222328
3,MT,13916.988281
4,WI,81781.779861
5,SD,14151.357901
6,AK,398576.219916
7,NE,7672.796780
8,MO,11283.318920
9,KS,5748.325735


In [15]:
geopypi.bar_chart(area_df, 'State', 'Area_SqKm', title='Wetland Area by State')